# Chaines de Markhov

In [51]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import numpy as np
from scipy import stats


table = pd.read_csv('Databases/swaphist.csv', delimiter = ";")

# Définition des fonctions

### Calcul de la matrice de transition

In [50]:
#Calcule la place de l'élément dans un des n groupes
def place(elem,table,freq):
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/freq
    return min(int((table[elem]-xmin)/delta),freq-1)

#Calcule la matrice de passage (fréquence de passage d'un groupe à l'autre)
def matrice(table, freq):
    trans = np.zeros((freq,freq))
    n = table.size
    for i in range(0,n-1):
        trans[(place(i,table,freq),place(i+1,table,freq))] = trans[(place(i,table,freq),place(i+1,table,freq))] + 1
    for i in range(0,freq):
        sum_ligne = np.sum(trans[i])
        #Certains intervalles ne sont jamais visités. Dans ce cas on décide qu'il y a une probabilité 1 d'y rester 
        if sum_ligne == 0 : 
            trans[(i,i)] = 0
        else :
            for j in range(0,freq):
                trans[(i,j)] = trans[(i,j)]/sum_ligne
    return trans

#La matrice de transition est-elle bien stochastique ?
test = matrice(var['1to5'].values,20)
for i in range(0,test.shape[0]):
    #print(sum(test[i]))
    if(sum(test[i])!=1.0): print(sum(test[i]))

0.0
0.0
0.0
0.0


Les matrices de transitions ne sont pas stochastiques car certains intervalles de valeurs ne sont jamais visités : typiquement, les valeurs élevés atteintes durant le pic de 2008-2013. Les variations sont très importantes lors de cette période et les volatilités font des sauts importants d'une semaine à l'autre.

Il faudra prendre cela en compte lorsque l'on conduira les tests. En effet, les tests doivent être effectués sur des matrices stochastiques. On supprimera donc les colonnes et lignes vides.

### Fonctions auxiliaires pour le calcul des statistiques de test

In [52]:
#Une fonction pour supprimer les colonnes et lignes qui ne contiennent que des zeros
def zero(mat):
    liste = []
    for i in range(0,mat.shape[0]-1):
        if(sum(mat[i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,0)
    liste = []
    for i in range(0,mat.shape[1]-1): 
        if(sum(mat[:,i])==0): liste.append(i)
    mat = scipy.delete(mat,liste,1)
    return mat

In [19]:
def A(table,i,freq,T):
    a=0
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/T
    tmp=0
    for t in range(0,T-1):
        sous_table = table[int(t*delta+xmin): int((t+1)*delta+xmin)]
        tmp =+ n(sous_table,i,freq)
        if n(sous_table,i,freq)!=0 : a+=1
    #print(tmp,n(table,table,i,freq))
    return a

def B(MT,i):
    B = []
    for j in range(0,MT.shape[1]-1):
        if MT[i,j]!=0 : B.append(j)
    return B

def n(sous_table,i,freq):
    tot = 0
    for elem in sous_table :
        if place(elem,sous_table,freq)==i : tot += 1
    return tot

def stat(table, sous_table, freq):
    dg_lbt = 0
    MT = matrice(table, freq)
    stat = 0
    sous_MT = matrice(sous_table, freq)
    for i in range(0,MT.shape[0]-1):
        ni = n(sous_table,i, freq)
        Bi = B(MT,i)
        for j in Bi:
            if MT[i,j]!=0 : stat += ni*(sous_MT[i,j]-MT[i,j])**2/MT[i,j]
    return stat

In [66]:
#La matrice de transition de la table est-elle homogène ?
def homog(table, freq=50, T=5):
    for i,elem in enumerate(table):
        if i>0:
            Q,p = homog_temp(table[elem].values,freq,T)
            m1 = homog_int(table[elem].values,freq,T)
            m2 = homog_reg(table[elem].values,freq,T)
            print("p-valeur : ",p, " - résultat du test par intervalles : ",m1," - résultat su test par régions : ",m2)
            rep = p>0.05 and m1>0.5 and m2>0.5
            if rep : print("La série de valeur pour le swaption ",elem," est homogène")
            else : print("La série de valeur pour le swaption ",elem," n'est pas homogène")
    return 0

## Les matrices de transition sont elles bien homogènes ?

Pour que la matrice de transition calculée à partir des fréquences de transition ait un sens, il fait s'assurer qu'elle est bien homogène en temps. On va donc mener plusieurs test pour vérifier cette hypothèse lourde.

### Test d'homogénéité des intervalles par rapport à la matrice de transition globale

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On vérifie que, si on calcule la matrice de transition sur chacun des intervalles, on retrouve bien la matrice de transition calculée sur toutes les valeurs de la série par un test du khi-2.


Notre hypothèse nulle est donc : $H_0 = \{\forall t, \le T, \forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [29]:
def homog_temp(table, freq, T=5):
    N = table.size
    Q = 0
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/T
    for t in range(0,T):
        sous_table = table[int(t*delta+xmin): int((t+1)*delta+xmin)]
        Q += stat(table,sous_table,freq)
    
    dg_lib = 0
    MT = matrice(table, freq)
    for i in range(0, MT.shape[0]-1):
        dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)

In [47]:
homog_temp(var['1to5'].values,50)

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(200.3324876314162, 0.0)

### Test d'homogénéité par intervalle

On subdivise notre série de valeurs en un nombre $T$ d'intervalles. On va effectuer $T$ tests pour vérifier si chacune des $T$ matrices de transitions sont bien les mêmes que la matrice de transition globale.


Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t <T$ : $H_0^t = \{\forall i,j, p_{i,j} (t) = p_{i,j}\}$

In [69]:
def homog_int(table, freq, T=5):
    N = table.size
    Q = 0
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/T
    moy = 0
    for t in range(0,T):
        sous_table = table[int(t*delta+xmin): int((t+1)*delta+xmin)]
        Q = stat(table,sous_table,freq)
        dg_lib = 0
        MT = matrice(table, freq)
        for i in range(0, MT.shape[0]-1):
            dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
            p = 1-stats.chi2.cdf(Q,dg_lib)
        #print("p-valeur du test sur le ",t,"-ième intervalle : ",p)
        if p>0.05 : moy += 1
    
    #print(moy," intervalles de temps sur ",T," sont homogènes")    
    return moy/T

In [48]:
homog_int(var['1to5'].values,50)

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


p-valeur du test sur le  0 -ième intervalle :  6.40796304907e-11
p-valeur du test sur le  1 -ième intervalle :  5.27122789862e-12
p-valeur du test sur le  2 -ième intervalle :  4.51504628129e-08
p-valeur du test sur le  3 -ième intervalle :  8.84269886599e-05
p-valeur du test sur le  4 -ième intervalle :  2.33598695942e-11
0  intervalles de temps sur  5  sont homogènes


0

### Test région contre région

On subdivise toujours notre série de valeurs en un nombre $T$ d'intervalles. Pour finir, on vérifie si les matrices de transitions de deux périodes sont identiques.

Notre hypothèse nulle est donc, pour chacun des intervalles $ 0\le t,u <T$ : $H_0^{t,u} = \{\forall i,j, p_{i,j}^t (t) = p_{i,j}^u\}$

In [68]:
def homog_reg(table, freq, T=5):
    N = table.size
    Q = 0
    xmin = min(table)
    xmax = max(table)
    delta = (xmax-xmin)/T
    moy = 0
    t,u = 0,0
    for t in range(0,T):
        for u in range(t+1,T):
            sous_table1 = table[int(t*delta+xmin): int((t+1)*delta+xmin)]
            sous_table2 = table[int(u*delta+xmin): int((u+1)*delta+xmin)]
            Q = stat(sous_table1,sous_table2,freq)
            dg_lib = 0
            MT = matrice(sous_table1, freq)
            for i in range(0, MT.shape[0]-1):
                dg_lib += max((len(B(MT,i))-1)*(A(table,i,freq,T)-1),0)
                p = 1-stats.chi2.cdf(Q,dg_lib)
            #print("p-valeur du test entre le ",t+1,"-ième et le ",u+1,"-ième intervalle : ",p)
            if p>0.05 : moy += 1
    
    #print(moy," paires d'intervalles de temps sur ",T," intervalles sont homogènes")    
    return 2*moy/(T*(T-1))

In [57]:
homog_reg(var['1to5'].values,50)

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


p-valeur du test entre le  1 -ième et le  2 -ième intervalle :  7.19088405284e-05
p-valeur du test entre le  1 -ième et le  3 -ième intervalle :  2.55754570733e-06
p-valeur du test entre le  1 -ième et le  4 -ième intervalle :  0.00839337729713
p-valeur du test entre le  1 -ième et le  5 -ième intervalle :  0.166808706085
p-valeur du test entre le  2 -ième et le  3 -ième intervalle :  0.000293985482538
p-valeur du test entre le  2 -ième et le  4 -ième intervalle :  6.85229179909e-06
p-valeur du test entre le  2 -ième et le  5 -ième intervalle :  0.999930498263
p-valeur du test entre le  3 -ième et le  4 -ième intervalle :  0.0064381477776
p-valeur du test entre le  3 -ième et le  5 -ième intervalle :  0.0101785609128
p-valeur du test entre le  4 -ième et le  5 -ième intervalle :  0.124349095748
3  paires d'intervalles de temps sur  5  intervalles sont homogènes


0.3

## Deux matrices de transition sont-elles égales ?

Maintenant, on peut tester si deux matrices de transition sont égales. On appliquera la statistique de test précédement définie aux deux matrices.

Notre hypothèse nulle est donc, si on note $(p_{i,j}^0)$ et $(p_{i,j}^1)$ les coefficents de chacune des deux matrices de transition : $H_0 = \{\forall i,j, p_{i,j}^0 = p_{i,j}^1\}$

In [31]:
def egales(table1, table2, freq=20):
    Q = stat(table1,table2,freq)
    
    dg_lib = 0
    MT = matrice(table1, freq)
    for i in range(0, MT.shape[0]-1):
        #print(len(B(MT,i)),A(table,i,freq,T))
        dg_lib += (len(B(MT,i))-1)
    return Q, 1-stats.chi2.cdf(Q,dg_lib)


In [15]:
print(egales(var['1to5'].values,var['1to10'].values))
print(egales(var['10to10'].values,var['10to20'].values))

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


14
(22.828486777395511, 0.063118428944444527)
24
(32.841886807409708, 0.10745383633510475)


# Tests sur toute la période 2001-2016

## Homogénéité

In [70]:
homog(table)

/Users/aurianeblarre/miniconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


p-valeur :  0.0  - résultat du test par intervalles :  0.4  - résultat su test par régions :  0.3
La série de valeur pour le swaption  1to5  n'est pas homogène
p-valeur :  0.0  - résultat du test par intervalles :  0.4  - résultat su test par régions :  1.0
La série de valeur pour le swaption  1to10  n'est pas homogène
p-valeur :  0.0  - résultat du test par intervalles :  0.8  - résultat su test par régions :  1.0
La série de valeur pour le swaption  1to20  n'est pas homogène
p-valeur :  3.48848283593e-09  - résultat du test par intervalles :  1.0  - résultat su test par régions :  1.0
La série de valeur pour le swaption  5to5  n'est pas homogène
p-valeur :  0.0  - résultat du test par intervalles :  0.8  - résultat su test par régions :  1.0
La série de valeur pour le swaption  5to10  n'est pas homogène
p-valeur :  0.00140298839367  - résultat du test par intervalles :  1.0  - résultat su test par régions :  1.0
La série de valeur pour le swaption  5to20  n'est pas homogène
p-valeur 

0

Comme on pouvait s'y attendre, les matrices de transition sur la période 2001-2016 ne sont homogène dans le temps pour aucun des swaptions. Nous allons donc restreindre notre études à de plus petits intervalles.

# Test sur la période 2001-2008